## 1. Test LLM and prompt types

In [ ]:
from configuration.llm_inference_configuration import APILLMConfiguration
from llm.azure_openai import AzureOpenAILLM
from prompt.zero_shot import ZeroShotPrompt
from prompt.few_shot import FewShotPrompt

In [ ]:
gpt_4o_azure_configuration = APILLMConfiguration()
gpt_4o_azure_configuration.load("configuration/yaml/llm/gpt_4o.yaml")
azure_gpt_4o = AzureOpenAILLM(gpt_4o_azure_configuration)

In [ ]:
sample_zero_shot_message = ZeroShotPrompt(
    [
        {
            "role": "user", 
            "content": "Health check. Say 'hi' to start the conversation."
        }
    ]
)

sample_zero_shot_message_responses = azure_gpt_4o.query(query=sample_zero_shot_message.prompt, num_responses=2)
print(azure_gpt_4o.get_response_texts(sample_zero_shot_message_responses))

In [ ]:
sample_few_shots_message = FewShotPrompt(
    [
        {
            "role": "system", 
            "content": "You are NetMind assistant. You task is to answer to the user anything about Viettel Group."
        },
        {
            "role": "user", 
            "content": "Tell me about Viettel Group."
        },
        {
            "role": "assistant", 
            "content": "Viettel Group is a Vietnamese multinational telecommunications company headquartered in Hanoi, Vietnam. It is a state-owned enterprise and operated by the Ministry of Defence. You can find out more about Viettel Group at https://viettel.vn/."
        },
        {
            "role": "user",
            "content": "What is the revenue of Viettel Group?"
        }
    ]
)

sample_few_shots_message_responses = azure_gpt_4o.query(query=sample_few_shots_message.prompt, num_responses=2)
print(azure_gpt_4o.get_response_texts(sample_few_shots_message_responses))

## 2. Test tools

### 2.1. Python code runner tool

In [1]:
from tools.python_code_runner import PythonCodeRunnerTool
from configuration.tool_configuration import ToolConfiguration

In [2]:
python_code_runner_config = ToolConfiguration()
python_code_runner_config.load("configuration/yaml/tools/python_code_runner.yaml")
python_code_runner = PythonCodeRunnerTool(python_code_runner_config)

In [4]:
input_code_list = [
    {
        "code_string": "a = 11\nb = 20\nresult = a + b\n\ndef greet(name):\n return f'Hello, {name}!'\n \nmessage = greet('World')",
        "id": "1"
    },
    {
        "code_string": "def multiply(x, y):\n\treturn x * y\n\nnum1 = 5\nnum2 = 3\nresult = multiply(num1, num2)\n\nprint(f'The product of {num1} and {num2} is {result}.')",
        "id": "2"
    }
]

results = python_code_runner.execute(input_code_list=input_code_list)
results

[{'id': '1',
  'result': {'a': 11, 'b': 20, 'result': 31, 'message': 'Hello, World!'}},
 {'id': '2', 'result': {'num1': 5, 'num2': 3, 'result': 15}}]

## 3. Test memory

In [9]:
def sieve(limit):
    if limit < 2:
        return []
    if limit == 2:
        return [False, True]
    if limit == 3:
        return [False, True, True]
    res = [False]
    if limit >= 2:
        res[2] = True
    if limit >= 3:
        res[3] = True
    for i in range(4, limit + 1):
        res[i] = False
    i = 1
    while i <= limit:
        j = 1
        while j <= limit:
            n = (4 * i * i) + (j * j)
            if (n <= limit and (n % 12 == 1 or
                                n % 12 == 5)):
                res[n] ^= True
            n = (3 * i * i) + (j * j)
            if n <= limit and n % 12 == 7:
                res[n] ^= True
            n = (3 * i * i) - (j * j)
            if (i > j and n <= limit and
                    n % 12 == 11):
                res[n] ^= True
            j += 1
        i += 1
    r = 5
    while r * r <= limit:
        if res[r]:
            for i in range(r * r, limit + 1, r * r):
                res[i] = False
        r += 1
    return res
def pick_prime(primes, min_size=1000):
    """returns a suitable prime to use as modulus"""
    for prime in primes:
        if prime >= min_size:
            return prime
    # if no prime large enough exists, use last one on list
    return primes[-1]
def hash(string, modulus):
    """implements polynomial rolling of string keys"""
    hash_value = 5381
    for char in string:
        # hash = 33 XOR ord(c)
        hash_value = ((hash_value << 5) + hash_value) ^ ord(char)
    return hash_value % modulus
if __name__ == '__main__':
    # generate primes list to use as modulus
    primes = sieve(10000) # modify limit based on your needs
    modulus = pick_prime(primes, 1000)
    test_array = ["alpha","beta","gamma","delta","epsilon"]
    for string in test_array:
        hash_value = hash(string, modulus)
        print(f"Hash of {string} is {hash_value}")

IndexError: list assignment index out of range